In [ ]:
import os
import matplotlib.pyplot as plt
from deepcell.utils.plot_utils import (
    create_rgb_image,
    make_outline_overlay,
)
import skimage
from skimage.io import imsave
from skimage import img_as_ubyte

from tissue_preparation import (
    read_image, 
    generate_nuclear_and_membrane,
    crop_out,
    stack_nuclear_and_membrane,
    run_segmentation,
)

In [ ]:
in_dir = "/fs/scratch/PCON0100/feng1426/temp/Benchmarking_tissue_preparation_data"
out_dir = "/fs/scratch/PCON0100/feng1426/temp/Benchmarking_tissue_preparation_data/out"

## Segmentation

### slide 1

In [ ]:
img1 = read_image(f"{in_dir}/Slide 1_20 min HIER 1h RT stain_Scan1.qptiff")
# use this to check which marker is which index in the qptiff
# indexing in python starts at 0 not at 1 like in R, so [0, :, :] = C1 in the qptiff
plt.figure(figsize = (5, 5))
plt.imshow(img1[0, :, :])

In [ ]:
nuclear1, membrane1 = generate_nuclear_and_membrane(img1)
# show the final nuclear and membrane arrays as images
fig, ax = plt.subplots(1, 2, figsize=(10, 10))
ax[0].imshow(nuclear1)
ax[1].imshow(membrane1)

ax[0].set_title('nuclear1')
ax[1].set_title('membrane1')

In [ ]:
# save nuclear and membrane tiffs for future reference 
# (can also check that nuclear is actually nuclear by opening the image, hard to tell in a python notebook)
imsave(f"{out_dir}/MESMER_outputs/nuclear1.tiff", nuclear1, check_contrast = False)
imsave(f"{out_dir}/MESMER_outputs/membrane1.tiff", membrane1, check_contrast = False)

In [ ]:
stacked_img1 = stack_nuclear_and_membrane(nuclear=nuclear1, membrane=membrane1)

In [ ]:
# crop out area of interest
y_min1 = 9500
y_max1 = 15000
x_min1 = 5000
x_max1 = 12000

fig, ax = plt.subplots(1, 2, figsize=(10, 10))
ax[0].imshow(stacked_img1[0, :, :, 0])
ax[1].imshow(stacked_img1[0, y_min1:y_max1, x_min1:x_max1, 0])

ax[0].set_title('full1')
ax[1].set_title('cropped1')

In [ ]:
cropped_stack1 = crop_out(stacked_img1, xmin=x_min1, xmax=x_max1, ymin=y_min1, ymax=y_max1)
cropped_stack1.shape

In [ ]:
# maxima_threshold controls what is considered a unique cell (lower values = more separate cells, higher values = fewer cells)
# interior_threshold determines what is considered background/not part of a cell (lower value = larger cells)
maxima_threshold = 0.075
interior_threshold = 0.2
predictions1 = run_segmentation(img=cropped_stack1, maxima_threshold=maxima_threshold, interior_threshold=interior_threshold)

In [ ]:
rgb_image = create_rgb_image(cropped_stack1, channel_colors = ["green", "blue"])
overlay = make_outline_overlay(rgb_data = rgb_image, predictions = predictions1)

# save MESMER outputs
output_dir = f'{out_dir}/MESMER_outputs/{maxima_threshold}maxima_1_{interior_threshold}interior_1/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

skimage.io.imsave(output_dir + "seg_outline1.tiff", img_as_ubyte(overlay[0, ..., 0]), check_contrast = False) # segmentation outline
skimage.io.imsave(output_dir + "seg_overlay1.tiff", img_as_ubyte(overlay[0, ...]), check_contrast = False) # segmentation overlay (nuc + membrane + outline)
skimage.io.imsave(output_dir + "MESMER_mask1.tiff", predictions1[0, ..., 0], check_contrast = False) # MESMER mask